<a href="https://colab.research.google.com/github/GoTudering/Final_Project_MTH3033/blob/main/Lenet_5_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Low Level**

In [ ]:
inputs = torch.Tensor(1,1,28,28)
print('Tensor shape : {}'.format(inputs.shape))

Tensor shape : torch.Size([1, 1, 28, 28])


In [ ]:
conv1 = nn.Conv2d(1,6,kernel_size = 5,padding= 2, stride = 1 )
print(conv1)

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))


In [ ]:
conv2 = nn.Conv2d(6,16,kernel_size = 5, stride = 1)
print(conv2)

Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))


In [ ]:
conv3 = nn.Conv2d(16,120,kernel_size = 5, stride = 1)
print(conv3)

Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))


In [ ]:
pool = nn.AvgPool2d(2,stride = 2)
print(pool)

AvgPool2d(kernel_size=2, stride=2, padding=0)


In [ ]:

out = conv1(inputs)
print(out.shape)

torch.Size([1, 6, 28, 28])


In [ ]:
out = pool(out)
print(out.shape)

torch.Size([1, 6, 14, 14])


In [ ]:
out = conv2(out)
print(out.shape)

torch.Size([1, 16, 10, 10])


In [ ]:
out = pool(out)
print(out.shape)

torch.Size([1, 16, 5, 5])


In [ ]:
out = conv3(out)
print(out.shape)

torch.Size([1, 120, 1, 1])


In [ ]:
out.size(0)

1

In [ ]:
out.size(1)

120

In [ ]:
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 120])


In [ ]:
fc1 = nn.Linear(120,84)
out = fc1(out)
print(out.shape)

torch.Size([1, 84])


In [ ]:
fc2 = nn.Linear(84,10)
out = fc2(out)
print(out.shape)

torch.Size([1, 10])


# **최종**

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 100

In [5]:
Fashion_mnist_train = dsets.FashionMNIST(root='FashionMNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

Fashion_mnist_test = dsets.FashionMNIST(root='FashionMNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [6]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=Fashion_mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
# CNN Model
class Lenet(torch.nn.Module):

    def __init__(self):
        super(Lenet, self).__init__()
        self.keep_prob = 0.5 # for dropout
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 6)
        #    Pool     -> (?, 14, 14, 6)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 6)
        #    Conv      ->(?, 10, 10, 16)
        #    Pool      ->(?, 5, 5, 16)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, kernel_size=5, stride=1),
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 5, 5, 16)
        #    Conv      ->(?, 1, 1, 120)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 120, kernel_size=5, stride=1),
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(1,stride = 1))

        # L4 FC 1x1x120 inputs -> 84 outputs
        self.fc1 = torch.nn.Linear(1 * 1 * 120, 84, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.Sigmoid(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 84 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(84, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [8]:
# instantiate CNN model
model = Lenet().to(device)

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 2.30245304
[Epoch:    2] cost = 0.972695947
[Epoch:    3] cost = 0.693885922
[Epoch:    4] cost = 0.623212397
[Epoch:    5] cost = 0.573274791
[Epoch:    6] cost = 0.538884699
[Epoch:    7] cost = 0.51257062
[Epoch:    8] cost = 0.490554631
[Epoch:    9] cost = 0.471953362
[Epoch:   10] cost = 0.455653012
[Epoch:   11] cost = 0.442127824
[Epoch:   12] cost = 0.429072648
[Epoch:   13] cost = 0.417727232
[Epoch:   14] cost = 0.409175992
[Epoch:   15] cost = 0.400433362
[Epoch:   16] cost = 0.39276734
[Epoch:   17] cost = 0.383944154
[Epoch:   18] cost = 0.378917366
[Epoch:   19] cost = 0.371140301
[Epoch:   20] cost = 0.363427758
[Epoch:   21] cost = 0.359794468
[Epoch:   22] cost = 0.35398984
[Epoch:   23] cost = 0.351800352
[Epoch:   24] cost = 0.344941616
[Epoch:   25] cost = 0.342905879
[Epoch:   26] cost = 0.337796867
[Epoch:   27] cost = 0.335869342
[Epoch:   28] cost = 0.333348364
[Epoch:   29] cost = 0.326170534
[Epoch:   

In [11]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = Fashion_mnist_test.test_data.view(len(Fashion_mnist_test), 1, 28, 28).float().to(device)
    Y_test = Fashion_mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.8366000056266785


/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


# parameters
learning_rate = 0.001
, training_epochs = 30
, batch_size = 100
, [Epoch:   30] cost = 0.323462397
, Accuracy: 0.8366000056266785

# parameters
learning_rate = 0.0001
, training_epochs = 30
, batch_size = 100
, [Epoch:   30] cost = 0.623933136
, Accuracy: 0.5525999665260315

In [13]:
# parameters
learning_rate = 0.0001
training_epochs = 30
batch_size = 100

In [14]:
# instantiate CNN model
model = Lenet().to(device)

In [15]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 2.39497805
[Epoch:    2] cost = 2.31842113
[Epoch:    3] cost = 2.30986357
[Epoch:    4] cost = 2.30571485
[Epoch:    5] cost = 2.30316854
[Epoch:    6] cost = 2.06571603
[Epoch:    7] cost = 1.47964895
[Epoch:    8] cost = 1.2263447
[Epoch:    9] cost = 1.08970022
[Epoch:   10] cost = 0.998242736
[Epoch:   11] cost = 0.933161914
[Epoch:   12] cost = 0.887539864
[Epoch:   13] cost = 0.851607442
[Epoch:   14] cost = 0.822056592
[Epoch:   15] cost = 0.796923995
[Epoch:   16] cost = 0.775072396
[Epoch:   17] cost = 0.754238248
[Epoch:   18] cost = 0.735512793
[Epoch:   19] cost = 0.723216832
[Epoch:   20] cost = 0.711005986
[Epoch:   21] cost = 0.698223293
[Epoch:   22] cost = 0.686553776
[Epoch:   23] cost = 0.676157534
[Epoch:   24] cost = 0.667972147
[Epoch:   25] cost = 0.659668386
[Epoch:   26] cost = 0.652582765
[Epoch:   27] cost = 0.642806351
[Epoch:   28] cost = 0.636498094
[Epoch:   29] cost = 0.630700886
[Epoch:   30] co

In [17]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = Fashion_mnist_test.test_data.view(len(Fashion_mnist_test), 1, 28, 28).float().to(device)
    Y_test = Fashion_mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.5525999665260315


# parameters
learning_rate = 0.01
, training_epochs = 30
, batch_size = 100
, [Epoch:   30] cost = 0.231829226
, Accuracy: 0.8578999638557434

In [18]:
# parameters
learning_rate = 0.01
training_epochs = 30
batch_size = 100

In [19]:
# instantiate CNN model
model = Lenet().to(device)

In [20]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 1.13151336
[Epoch:    2] cost = 0.536779404
[Epoch:    3] cost = 0.45770672
[Epoch:    4] cost = 0.414605141
[Epoch:    5] cost = 0.389588058
[Epoch:    6] cost = 0.370925039
[Epoch:    7] cost = 0.349725723
[Epoch:    8] cost = 0.338615268
[Epoch:    9] cost = 0.326311082
[Epoch:   10] cost = 0.316673011
[Epoch:   11] cost = 0.309342921
[Epoch:   12] cost = 0.305294544
[Epoch:   13] cost = 0.29808566
[Epoch:   14] cost = 0.290970802
[Epoch:   15] cost = 0.285872281
[Epoch:   16] cost = 0.276337892
[Epoch:   17] cost = 0.275457233
[Epoch:   18] cost = 0.271694392
[Epoch:   19] cost = 0.266388208
[Epoch:   20] cost = 0.260963947
[Epoch:   21] cost = 0.259329498
[Epoch:   22] cost = 0.256788671
[Epoch:   23] cost = 0.253836364
[Epoch:   24] cost = 0.252434939
[Epoch:   25] cost = 0.246303022
[Epoch:   26] cost = 0.241216257
[Epoch:   27] cost = 0.239887923
[Epoch:   28] cost = 0.242459029
[Epoch:   29] cost = 0.23643057
[Epoch:   

In [22]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = Fashion_mnist_test.test_data.view(len(Fashion_mnist_test), 1, 28, 28).float().to(device)
    Y_test = Fashion_mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.8578999638557434
